# Principal Component Analysis

Think of the predictors in your dataset as dimensions in what we can usefully call "feature space". If we're predicting house prices, then we might have a 'square feet' dimension or a 'number of bathrooms' dimension, etc. Then each record (of a house or a house sale, say) would be represented as a point (or vector) in this feature space. Some would score higher on the 'latitude' dimension or lower on the 'number of bedrooms' dimension, or whatever.

One difficulty is that, despite our working nomenclature, these things aren't really _dimensions_ in the truest sense, since they're not independent of each other. When we talk about the x-, y-, and z-dimensions of Euclidean 3-space, for example, one important feature is that values of x have no bearing (per se) on values of y or of z. I can move three units along the x-dimension without changing my y- or z-position.

But the analogous thing is generally not true for datasets. When I increase my position along the 'number of bedrooms' dimension--or, better, _direction_, I also tend to increase my position along, say, the 'square feet' direction as well.

This is problematic for a couple reasons: One is that my model could be in effect "double-counting" certain features of my signal, which can lead to overfit models. And if my goal is inference or explanation, then I'm going to have a very hard time distinguishing between the idea that the number of bedrooms is what's _really_ predictive of housing  prices and the idea that the number of square feet is what's really so predictive.

The idea behind PCA is to transform our dataset into something more useful for building models. What we want to do is to build new dimensions (predictors) out of the dimensions we are given in such a way that:

(1) each dimension we draw captures as much of the remaining variance among our predictors as possible; and <br/>
(2) each dimension we draw is orthogonal to the ones we've already drawn.

## Motivation

Think back to multiple linear regression for a moment.

The fundamental idea is that I can get a better prediction for my dependent variable by considering a *linear combination of my predictors* than I can get by considering any one predictor by itself.

$\rightarrow$ **PCA insight**: If the combinations of predictors work better than the predictors themselves, then let's just treat the combinations as our primary dimensions!

But one problem with having lots of predictors is that it raises the chance that some will be nearly *collinear*.

$\rightarrow$ **PCA insight**: Since we're reconstructing our dimensions anyway, let's make sure that the dimensions we construct are mutually orthogonal! <br/>
$\rightarrow$ **PCA insight**: Moreover, since we'll be capturing much of the variance among our predictors in the first few dimensions we construct, we'll be able in effect to *reduce  the dimensionality* of our problem. Thus PCA is a fundamental tool in *dimensionality reduction*.

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [54]:
cars = pd.read_csv('/Users/gdamico/Desktop/cars.csv')

In [28]:
X_train, X_test, y_train, y_test = train_test_split(cars.drop('mpg', axis=1),
                                                    cars['mpg'],
                                                   random_state=20)

In [29]:
X_train[' cubicinches'].replace(' ', np.nan, inplace=True)
X_train[' cubicinches'] = X_train[' cubicinches'].map(float)
X_train[' cubicinches'].fillna(X_train[' cubicinches'].mean(skipna=True), inplace=True)

In [30]:
X_train[' weightlbs'].replace(' ', np.nan, inplace=True)
X_train[' weightlbs'] = X_train[' weightlbs'].map(float)
X_train[' weightlbs'].fillna(X_train[' weightlbs'].mean(), inplace=True)

In [31]:
X_train[' cylinders'] = X_train[' cylinders'].map(float)
X_train[' hp'] = X_train[' hp'].map(float)
X_train[' time-to-60'] = X_train[' time-to-60'].map(float)
X_train[' year'] = X_train[' year'].map(float)

In [32]:
# Scaling

ss = StandardScaler()

In [33]:
# Scale-transforming

X_tr_sc = ss.fit_transform(X_train.drop(' brand', axis=1))

In [55]:
# Let's construct a linear regression

lr = LinearRegression().fit(X_tr_sc, y_train)

# Score on train
lr.score(X_tr_sc, y_train)

In [35]:
def clean(df):
    for col in [' cubicinches', ' weightlbs']:
        df[col].replace(' ', np.nan, inplace=True)
        df[col] = df[col].map(float)
        df[col].replace(np.nan, df[col].mean(), inplace=True)
    return df

In [36]:
def to_float(df):
    for col in [' cylinders', ' hp', ' time-to-60', ' year']:
        df[col] = df[col].map(float)
    return df

In [37]:
def drop(df):
    return df.drop(' brand', axis=1)

In [38]:
def scale(df):
    return ss.transform(df)

In [39]:
test_cleaned = clean(X_test)
test_floated = to_float(test_cleaned)
test_dropped = drop(test_floated)
test_scaled = scale(test_dropped)

In [56]:
# Score on test

lr.score(test_scaled, y_test)

In [60]:
# Get the coefficients of the best-fit hyperplane

lr.coef_

Thus, our best-fit hyperplane is given by:

$-1.409\times cyl + 0.681\times in^3 - 0.480\times hp - 4.658\times lbs. -  0.176\times time_{60} + 2.427\times yr$

## Eigenvalues and Eigenvectors

The key idea is to diagonalize (i.e. find the eigendecomposition of) the covariance matrix. The decomposition will produce a set of orthogonal vectors that explain as much of the remaining variance as possible.

Let's say a word about eigenvalues and eigenvectors. It turns out that eigenvalues and -vectors have a dizzying number of applications. But the basic idea is that, if we can split a bunch of vectors (i.e. a matrix) into a set of mutually orthogonal vectors, then we can isolate the force of the bunch into discrete bits, each of which by itself acts like a simple linear transformation.

That's why the definition of an eigenvector is as it is: $\vec{x}$ is an eigenvector of the matrix $A$ if $A\vec{x} = \lambda\vec{x}$, for some scalar $\lambda$. That is, the vector is oriented in just such a direction that multiplying the matrix by it serves only to lengthen or shorten it.

Let's do a simple example.

Suppose we have the matrix
$A =
\begin{bmatrix}
5 & 3 \\
3 & 5 \\
\end{bmatrix}
$

Let's calculate the eigendecomposition of this matrix.

In [56]:
# We can use np.linalg.eig()

A = np.array([[5, 3], [3, 5]])
np.linalg.eig(A)

In [ ]:
v, q = np.linalg.eig(A)

In [54]:
# np.diag()

np.diag(v)

In [55]:
# Reconstruct A by multiplication

q.dot(np.diag(v)).dot(q.T)

## PCA by Hand
[Here's](https://www.youtube.com/watch?v=_UVHneBUBW0) a video introduction to PCA.

N.B.: What follows is indebted to http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda

In [61]:
# We'll start by producing the covariance matrix for the columns of X_tr_sc.

cov_mat = np.cov(X_tr_sc, rowvar=False)
cov_mat.shape

In [62]:
# np.linalg.eig(X) returns a double of NumPy arrays, the first containing
# the eigenvalues of X and the second containing the eigenvectors of X.

np.linalg.eig(cov_mat)

In [58]:
# Let's assign the results of eig(cov_mat) to a double of variables.

eigvals, eigvecs = np.linalg.eig(cov_mat)

In [59]:
# We'll now pair up each eigenvalue with its corresponding eigenvector.

eigpairs = [(eigvals[i], eigvecs[:, i]) for i in range(len(eigvals))]

In [60]:
# Let's look at the first element of eigpairs.



In [61]:
# The second element of each element in eigpairs is
# an eigenvector of the covariance matrix.



In [62]:
# We want to isolate the eigenvectors and create a matrix
# with a column for each. We'll use just three of these,
# corresponding to taking the first three principal components.

pcabh = np.hstack((eigpairs[0][1].reshape(6, 1),
                 eigpairs[1][1].reshape(6, 1),
                  eigpairs[2][1].reshape(6, 1)))
pcabh

In [63]:
# Now we simply want the dot-product of
# X_tr_sc with this matrix of the eigenvectors
# of the covariance matrix of X.

X_tr_sc.dot(pcabh)

In [64]:
# Naturally, sklearn has a shortcut for this!



In [65]:
# Let's check out the explained variance



In [66]:
# The ratio is often more informative



In [67]:
# We can also check out the Principal Components themselves



In [68]:
# Recall the columns of cars



The results of our PCA are as follows:

**PC1** = 0.454 * cylinders + 0.470 * cubicinches + 0.462 * hp + 0.440 * weightlbs - 0.357 * time-to-60 - 0.196 * year

**PC2** = -0.143 * cylinders - 0.110 * cubicinches - 0.023 * hp - 0.217 * weightlbs - 0.102 * time-to-60 - 0.954 * year

**PC3** = 0.204 * cylinders + 0.153 * cubicinches - 0.129 * hp + 0.361 * weightlbs + 0.860 * time-to-60 - 0.220 * year

It turns out that these loadings are encoded in the eigenvectors of $X^TX$. Notice that:

- the absolute values of the components of PC1 are the first components of the eigenvectors below, <br/>
- the absolute values of the components of PC2 are the second components of the eigenvectors below, <br/>
- etc. <br/>

We'll have more to say about this when we examine the singular value decomposition of matrices in Mod 4.

In [3]:
np.linalg.eig(X_tr_sc.T.dot(X_tr_sc))

## Normality

In [69]:
# These principal components should be normalized.
# If they are, then the sum of the squares of the
# loadings should be 1. Let's check!



## Orthogonality

In [70]:
# These principal components should also be
# mutually orthogonal. If they are, then the
# dot product of any two of them should be 0.
# Let's check!



## Visualizations

In [4]:
from matplotlib import pyplot as plt
%matplotlib inline

fig, ax = plt.subplots()
ax.plot(X_train_new[:, 0], y_train, 'r.');

In [5]:
fig, ax = plt.subplots()
ax.plot(X_train_new[:, 1], y_train, 'g.');

In [6]:
fig, ax = plt.subplots()
ax.plot(X_train_new[:, 2], y_train, 'k.');

Question: Is the first principal component the same line we would get if we constructed an ordinary least-squares regression line?

The answer is NO! Check out this post for an illuminating discussion: https://shankarmsy.github.io/posts/pca-vs-lr.html

## Modeling with New Dimensions

Now that we have optimized our features, we can build a new model with them!

In [7]:
lr_pca = LinearRegression()
lr_pca.fit(X_train_new, y_train)
lr_pca.score(X_train_new, y_train)

In [45]:
X_test_new = pca.transform(X_te_sc)

In [8]:
lr_pca.score(X_test_new, y_test)

In [9]:
lr_pca.coef_

Thus, our best-fit hyperplane is given by:

$-3.00\times PC1 - 1.15\times PC2 -2.49\times PC3$

Of course, since the principal components are just linear combinations of our original predictors, we could re-express this hyperplane in terms of those original predictors!

And if the PCA was worth anything, we should expect the new linear model to be *different from* the first!

Recall that we had:

**PC1** = 0.454 * cylinders + 0.470 * cubicinches + 0.462 * hp + 0.440 * weightlbs - 0.357 * time-to-60 - 0.196 * year

**PC2** = -0.143 * cylinders - 0.110 * cubicinches - 0.023 * hp - 0.217 * weightlbs - 0.102 * time-to-60 - 0.954 * year

**PC3** = 0.204 * cylinders + 0.153 * cubicinches - 0.129 * hp + 0.361 * weightlbs + 0.860 * time-to-60 - 0.220 * year

Therefore, our new PCA-made hyperplane can be expressed as:

$-3.00\times(0.454\times cyl + 0.470\times in^3 + 0.462\times hp + 0.440\times lbs. - 0.357\times time_{60} - 0.196\times yr)$ <br/> $- 1.15\times(-0.143\times cyl - 0.110\times in^3 - 0.023\times hp - 0.217\times lbs. - 0.102\times time_{60} - 0.954\times yr)$ <br/> $- 2.49\times(0.204\times cyl + 0.153\times in^3 - 0.129\times hp + 0.361\times lbs. + 0.860\times time_{60} - 0.219\times yr)$ <br/><br/> $= - 1.706\times cyl - 1.664\times in^3 -1.038\times hp - 1.969\times lbs. - 0.953\times time_{60} + 2.230\times yr$

Our first linear regression model had:

$-1.409\times cyl + 0.681\times in^3 - 0.480\times hp - 4.658\times lbs. -  0.176\times time_{60} + 2.427\times yr$,

which is clearly a different hyperplane.